###Цель проектной работы: обучить модель предсказывать тариф для пользователя мобильнйо связи по имеющимся о нем данным.
###Ход работы:
1. Открытие и изучение данных
2. Деление данных на выборки
3. Исследование модели
4. Тестирование модели на тестовой выборке

#1.Открытие и изучение данных

In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/users_behavior.csv')

In [ ]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


#2.Деление данных на выборки

In [ ]:
features = data.drop('is_ultra', axis = 1)
target = data['is_ultra']

In [ ]:
from sklearn.model_selection import train_test_split
features_remaining, features_test, target_remaining, target_test = train_test_split(features,
                                                                                    target,
                                                                                    test_size = 0.20,
                                                                                    random_state = 12345)#отрезаем тестовую выборку
features_train, features_valid, target_train, target_valid = train_test_split(features_remaining,
                                                                              target_remaining,
                                                                              test_size = 0.25,
                                                                              random_state = 12345)#делим оставшийся датасет на тренировочную и валидационную выборки

#3.Исследование моделей

##Первая модель - дерево решений

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier()
tree.fit(features_train, target_train)
tree_pred = tree.predict(features_valid)
print("Accuracy:",accuracy_score(tree_pred, target_valid))

Accuracy: 0.7169517884914464


В результате обучения и тестирования модели на валидационной выборке точность предсказания составила 0.71, что меньше, чем 0.75. Такая модель не подходит.

##Вторая модель - логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression
regression = LogisticRegression(random_state = 12345, solver = 'lbfgs', max_iter = 100)
regression.fit(features_train, target_train)
regression_pred = regression.predict(features_valid)
print("Accuracy:",accuracy_score(regression_pred, target_valid))

Accuracy: 0.7262830482115086


Логистическая регрессия дает точность 0.72, что не намного больше, чем предыдущая. Такая модель тоже не соответствует требованиям задания.


##Третья модель - лес решений

Для данной модели рассмотрим разные значения гиперпараметров: критерия, максимальной глубины дерева и количества деревьев.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
max_acc = 0
max_est = 0
best_parameters = []
criterion_list = ['gini', 'entropy']
for c in criterion_list:
  for depth in range(10,101,10):
    for est in range(10,101,10):
      forest = RandomForestClassifier(criterion = c, max_depth = depth, n_estimators = est, random_state=42)
      forest.fit(features_train, target_train)
      forest_pred = forest.predict(features_valid)
      acc = accuracy_score(forest_pred, target_valid)
      if acc > max_acc:
        max_acc = acc
        max_est = est
        best_parameters.append(c)
        best_parameters.append(depth)
        best_parameters.append(est)
print("Max accuracy:",max_acc, "| number of estimators:", best_parameters[2], '| maximum depth:', best_parameters[1], '| best criteria:', best_parameters[0])

Max accuracy: 0.8055987558320373 | number of estimators: 10 | maximum depth: 10 | best criteria: gini


Эта модель показала точность в 0.8, что удовлетворяет требования задания. Проверим ее на тестовой выборке.

#4.Проверка модели на тестовой выборке

In [ ]:
model = RandomForestClassifier(criterion = 'gini', max_depth = 10, n_estimators = 10, random_state=42)
model.fit(features_remaining, target_remaining)
predictions = accuracy_score(model.predict(features_test), target_test)
print(predictions)


0.7962674961119751


Модель работает исправно.

#5.Вывод
В результате проведенной работы были рассмотрены три модели: дерево решений, логистическая регрессия и лес решений. Первые две дали неудовлетворительную точность. Для третьей были подобраны параметры, при которых точность предсказания получается наилучшей. В итоге, точность предсказания при работе модели с валидационной выборкой составила 0.8, а при работе с тестовой выборкой - 0.79, что удовлетворяет требованию задания.